In [3]:
import os
import pandas as pd
from ast import literal_eval

In [5]:
df = pd.read_csv("data/dataset/entries_dataset.csv")

In [7]:
def _custom_eval(x):
    try:
        return literal_eval(x)
    except:
        return x


In [25]:
def _format_interviewee(interviewees):
    if len(interviewees) > 0 and isinstance(interviewees[0], dict):
        output = ""
        for interviewee in interviewees:
            name = interviewee["name"]
            role = interviewee["role"]
            organization = interviewee["organization"]
            location = interviewee["location"]
            location_type = interviewee["location_type"]
            gender = interviewee["gender"]
            organization_type = interviewee["organisation_type"]
            output += f"{name} ({gender}, {role}, {organization}, {organization_type}, {location_type}, {location})".replace(", -", "")
            output += " // "
        output = output[:-4]
        return output
    else:
        return "**Interviewees Not Found**"

In [26]:
for col in ["Framework Probabilities", "Secondary Tags", "Interviewee", "Document Source"]:
    df[col] = df[col].apply(_custom_eval)
    
    
df["Document Source"] = df["Document Source"].apply(lambda x: ", ".join(x))
df["Interviewee"] = df["Interviewee"].apply(_format_interviewee)

In [27]:
df

,Entry Type,entry_fig_path,Document Publishing Date,Document Source,Document Title,Interviewee,Number of Pages,File Name,project_name,sub_project_name,docs_name,doc_id,Extracted Entries,entry_id,Embeddings,Secondary Tags,Framework Probabilities
0,PDF Text,-,01/01/2015,"Rebecca Skovbye, Mark Gordon",Research and Knowledge Management Service (APPR),"- (M, Assurance Project Manager, Ethiopia, Ope...",2,Research and Knowledge Management Service.docx,Individual Interview notes,General,data/docs/Individual Interview notes/General/R...,0,Biometrics are just a way to identify individu...,0,"[-0.0396728515625, 0.07012939453125, 0.0172271...",[Severity->Issue Of Concern],{'What factors affect WFP's performance on tar...
1,PDF Text,-,01/01/2015,"Rebecca Skovbye, Mark Gordon",Research and Knowledge Management Service (APPR),"- (M, Assurance Project Manager, Ethiopia, Ope...",2,Research and Knowledge Management Service.docx,Individual Interview notes,General,data/docs/Individual Interview notes/General/R...,0,"If you have fingerprints, this does not help t...",1,"[-0.09735107421875, 0.03485107421875, -0.02499...","[Displaced->Migrants, Displaced->Refugees, Dis...",{'How effectively does WFP engage and collabor...
2,PDF Text,-,01/01/2015,"Rebecca Skovbye, Mark Gordon",Research and Knowledge Management Service (APPR),"- (M, Assurance Project Manager, Ethiopia, Ope...",2,Research and Knowledge Management Service.docx,Individual Interview notes,General,data/docs/Individual Interview notes/General/R...,0,"Linked to outcome indicators on food security,...",2,"[0.0738525390625, -0.003726959228515625, 0.028...",[Severity->Severe Issue],{'What factors affect WFP's performance on tar...
3,PDF Text,-,01/01/2015,"Rebecca Skovbye, Mark Gordon",Research and Knowledge Management Service (APPR),"- (M, Assurance Project Manager, Ethiopia, Ope...",2,Research and Knowledge Management Service.docx,Individual Interview notes,General,data/docs/Individual Interview notes/General/R...,0,Yemen – 3 years of 50.000 HH studies..,3,"[0.1082763671875, 0.1331787109375, -0.02101135...",[Severity->Minor Issue],{'How relevant and appropriate are WFP's appro...
4,PDF Text,-,01/01/2015,"Rebecca Skovbye, Mark Gordon",Research and Knowledge Management Service (APPR),"- (M, Assurance Project Manager, Ethiopia, Ope...",2,Research and Knowledge Management Service.docx,Individual Interview notes,General,data/docs/Individual Interview notes/General/R...,0,These are used for livelihood.. But you can us...,4,"[0.0289764404296875, 0.043853759765625, 0.0176...","[Displaced->Migrants, Displaced->Refugees, Dis...",{'How relevant and appropriate are WFP's appro...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1993,PDF Text,-,-/01/2013,Joël Glasman,Humanitarianism and the Quantification of Huma...,"Joël Glasman (M, Other) // Marc Dubois (M, Oth...",2,Joel Glasman.pdf,Individual Interview notes,External stakeholder,data/docs/Individual Interview notes/External ...,59,"– besteht nicht nur auf ein paar Indikatoren,,...",1993,"[-0.07830810546875, 0.05401611328125, -0.02471...",[],{'What factors affect WFP's performance on tar...
1994,PDF Text,-,-/01/2013,Joël Glasman,Humanitarianism and the Quantification of Huma...,"Joël Glasman (M, Other) // Marc Dubois (M, Oth...",2,Joel Glasman.pdf,Individual Interview notes,External stakeholder,data/docs/Individual Interview notes/External ...,59,"Das ist leicht zu sagen und zu verstehen,, abe...",1994,"[-0.0200653076171875, 0.030853271484375, -0.06...",[Non Displaced->Host],{'How relevant and appropriate are WFP's appro...
1995,PDF Text,-,-/01/2013,Joël Glasman,Humanitarianism and the Quantification of Huma...,"Joël Glasman (M, Other) // Marc Dubois (M, Oth...",2,Joel Glasman.pdf,Individual Interview notes,External stakeholder,data/docs/Individual Interview notes/External ...,59,Institutionalisierung ist die Hauptfrage... Ma...,1995,"[0.06939697265625, 0.033966064453125, -0.12036...",[],{'What factors affect WFP's performance on

In [28]:
framework_questions = sorted(list(df["Framework Probabilities"].iloc[0].keys()))
framework_questions


['How effectively does WFP engage and collaborate with others on targeting and prioritization?->How well does WFP coordinate its targeting and prioritization practices with those of partners in relevant clusters and the Humanitarian Country Team and what role do joint targeting approaches play in this context?->Engagement with cluster and other humanitarian and resilience partners on targetting and prioritization',
 'How effectively does WFP engage and collaborate with others on targeting and prioritization?->How well does WFP coordinate its targeting and prioritization practices with those of partners in relevant clusters and the Humanitarian Country Team and what role do joint targeting approaches play in this context?->Experiences with joint targeting approaches with other UN agencies',
 'How effectively does WFP engage and collaborate with others on targeting and prioritization?->How well does WFP coordinate its targeting and prioritization practices with those of partners in relev

In [40]:
projects = sorted(list(df["project_name"].unique()))
projects

['Individual Interview notes']

In [39]:
subprojects = sorted(list(df["sub_project_name"].unique()))
subprojects

['External stakeholder', 'General', 'RBs']

In [44]:
final_shown_df = pd.DataFrame()

for one_project in projects:
    for one_subproject in subprojects:
        for one_question in framework_questions:

            pillar = one_question.split("->")[0]
            subpillar = one_question.split("->")[1]
            indicator = one_question.split("->")[2]
            one_df = df.copy()
            one_df["Relevance"] = one_df["Framework Probabilities"].apply(
                lambda x: x[one_question]
            )
            one_df = one_df[
                (one_df["Relevance"] > 0.6)
                & (one_df["project_name"] == one_project)
                & (one_df["sub_project_name"] == one_subproject)
            ].sort_values(by="Relevance", ascending=False)
            one_df["Relevance"] = one_df["Relevance"].apply(lambda x: f"{100*x:.0f}%")
            one_df["Pillar"] = pillar
            one_df["Subpillar"] = subpillar
            one_df["Indicator"] = indicator
            one_df = one_df[
                [
                    "Pillar",
                    "Subpillar",
                    "Indicator",
                    "project_name",
                    "sub_project_name",
                    "Extracted Entries",
                    "File Name",
                    "Document Publishing Date",
                    "Document Source",
                    "Document Title",
                    "Interviewee",
                    "Number of Pages",
                    "Relevance",
                ]
            ].rename(
                columns={
                    "Extracted Entries": "Extract",
                    "project_name": "Project",
                    "sub_project_name": "Sub-Project",
                }
            )
            final_shown_df = pd.concat([final_shown_df, one_df])
            # add empty row
            empty_row = pd.DataFrame([{col: "" for col in final_shown_df.columns}])
            final_shown_df = pd.concat([final_shown_df, empty_row], ignore_index=True)


In [45]:
final_shown_df.tail(30)

,Pillar,Subpillar,Indicator,Project,Sub-Project,Extract,File Name,Document Publishing Date,Document Source,Document Title,Interviewee,Number of Pages,Relevance
6326,,,,,,,,,,,,,
6327,general_questions,What are the factors interview discuss as infl...,"Which are positive factors, i.e. which enable ...",Individual Interview notes,RBs,Main interest is that budget is defendable and...,RBP.docx,September 2024,"Regional Bureau for Latin America, Regional As...",Discussion Report - Regional Bureau for Latin ...,"Marta Ortiz (F, Regional vulnerability and map...",3,94%
6328,general_questions,What are the factors interview discuss as infl...,"Which are positive factors, i.e. which enable ...",Individual Interview notes,RBs,Main interest is that budget is defendable and...,RBP.pdf,September 2024,"Regional Bureau for Latin America, Regional As...",Discussion Report - Regional Bureau for Latin ...,"Marta Ortiz (F, Regional vulnerability and map...",3,94%
6329,general_questions,What are the factors interview discuss as infl...,"Which are positive factors, i.e. which enable ...",Individual Interview notes,RBs,"But for program advisors, with smaller pay gra...",RBP.docx,September 2024,"Regional Bureau for Latin America, Regional As...",Discussion Report - Regional Bureau for Latin ...,"Marta Ortiz (F, Regional vulnerability and map...",3,91%
6330,general_questions,What are the factors interview discuss as infl...,"Which are positive factors, i.e. which enable ...",Individual Interview notes,RBs,"But for program advisors, with smaller pay gra...",RBP.pdf,September 2024,"Regional Bureau for Latin America, Regional As...",Discussion Report - Regional Bureau for Latin ...,"Marta Ortiz (F, Regional vulnerability and map...",3,91%
6331,general_questions,What are the factors interview discuss as infl...,"Which are positive factors, i.e. which enable ...",Individual Interview notes,RBs,"We have a lot to do in the area,, but not enou...",RBD.docx,14/03/2023,RBD,Elias’ follow-up interview with Rachida,"Rachida Aouameur (F, Emergency Coordination an...",5,85%
6332,general_questions,What are the factors interview discuss as infl...,"Which are positive factors, i.e. which enable ...",Individual Interview notes,RBs,The use of CBT is to check which model is help...,RBD.docx,14/03/2023,RBD,Elias’ follow-up interview with Rachida,"Rachida Aouameur (F, Emergency Coordination an...",5,83%
6333,general_questions,What are the factors interview discuss as infl...,"Which are positive factors, i.e. which enable ...",Individual Interview notes,RBs,"... Corporate Strategic Plan talks,,, for inst...",RBP.docx,September 2024,"Regional Bureau for Latin America, Regional As...",Discussion Report - Regional Bureau for Latin ...,"Marta Ortiz (F, Regional vulnerability and map...",3,79%
6334,general_questions,What are the factors interview discuss as infl...,"Which are positive factors, i.e. which enable ...",Individual Interview notes,RBs,"... Corporate Strategic Plan talks,,, for inst...",RBP.pdf,September 2024,"Regional Bureau for Latin America, Regional As...",Discussion Report - Regional Bureau for Latin ...,"Marta Ortiz (F, Regional vulnerability and map...",3,79%
6335,general_questions,What are the factors interview discuss as infl...,"Which are positive factors, i.e. which enable ...",Individual Interview notes,RBs,"Consistent data collection.., Declining fundin...",RBC.docx,-/01/2024,"-, -",RBC,"Tobias Flaemig (M, head of knowledge creation ...",3,78%


In [46]:
final_shown_df.to_csv("data/dataset/final_shown_df.csv", index=False)